<a href="https://colab.research.google.com/github/inspire007/KaggleDatasets/blob/main/Titanic_Machine_Learning_from_Disaster_Kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score

data = pd.read_csv('train.csv')

#split X,y
X = data.iloc[:, [2,4,5,6,7,11]].values
y = data.iloc[:, 1].values

#encode categorical values
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1,5])], remainder='passthrough');
X = ct.fit_transform(X)

#replace any missing values or NaN by mean of the column
si = SimpleImputer(missing_values=np.nan, strategy='mean')
X = si.fit_transform(X)

#for testing purpose split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

#scaling needed since regression models are not used
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

#random forest
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=200, random_state=42)
rfc.fit(X_train, y_train)

y_pred = rfc.predict(X_test)

#print(np.concatenate((
#    y_pred.reshape(len(y_pred),1),
#    y_test.reshape(len(y_test),1)
#),1))

cf = confusion_matrix(y_test, y_pred)
ac = accuracy_score(y_test, y_pred)

print("Random forest: \n", cf, "\n", ac, "\n")


#decision tree
from sklearn.tree import DecisionTreeClassifier
rfc = DecisionTreeClassifier(random_state=42)
rfc.fit(X_train, y_train)

y_pred = rfc.predict(X_test)


cf = confusion_matrix(y_test, y_pred)
ac = accuracy_score(y_test, y_pred)

print("Decision tree: \n", cf, "\n", ac, "\n")


#support vector model
from sklearn.svm import SVC

sv = SVC(kernel='rbf', random_state=42)
sv.fit(X_train, y_train)

y_pred = sv.predict(X_test)

cf = confusion_matrix(y_test, y_pred)
ac = accuracy_score(y_test, y_pred)

print("SVC: \n", cf, "\n", ac, "\n")

#importing test data
data = pd.read_csv('test.csv')
X_test_actual = data.iloc[:, [1,3,4,5,6,10]].values

#column transfer
X_test_actual = ct.transform(X_test_actual)

#handle missing values
X_test_actual = si.transform(X_test_actual)

#scaling
sc = StandardScaler()
X = sc.fit_transform(X)
X_test_actual = sc.transform(X_test_actual)

#train full model
model = SVC(random_state=42)
model.fit(X, y)
y_pred_actual = model.predict(X_test_actual)
#convert to column
y_pred_actual = y_pred_actual.reshape(len(y_pred_actual), 1)

#print(y_pred_actual, len(y_pred_actual))
pid = data['PassengerId'].values
pid = pid.reshape(len(pid), 1)

output = np.concatenate((pid,y_pred_actual), 1)
np.savetxt('output.csv', output, header="PassengerId,Survived", fmt ="%d", delimiter=',', comments='')

Random forest: 
 [[92 13]
 [19 55]] 
 0.8212290502793296 

Decision tree: 
 [[89 16]
 [24 50]] 
 0.776536312849162 

SVC: 
 [[95 10]
 [22 52]] 
 0.8212290502793296 

